In [ ]:
import numpy as numpy
import pandas as pd

In [ ]:
data=pd.read_csv('/content/train.csv')

In [ ]:
data=data.drop(['id','keyword','location'],axis=1)

In [ ]:
data.head(5)

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
data.isnull().sum()

text      0
target    0
tokens    0
vec       0
dtype: int64

In [ ]:
!pip install --upgrade gensim -q
!pip install matplotlib -q

In [ ]:
import gensim
from gensim.models import word2vec

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import string


/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
gensim.__version__

'4.3.1'

In [ ]:
np.random.seed(42)

In [ ]:
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
wv=api.load('glove-twitter-50')

[=================================================-] 99.7% 198.9/199.5MB downloaded


In [ ]:
type(wv)

gensim.models.keyedvectors.KeyedVectors

In [ ]:
wv['mango']

array([-0.86458 , -0.6638  , -0.26769 , -0.018869,  0.25331 ,  0.43483 ,
       -0.47727 , -0.16365 ,  0.01589 , -0.30916 , -1.1219  , -0.51062 ,
       -2.1855  ,  0.13349 , -0.071669, -0.14381 ,  0.96755 , -0.37408 ,
        0.62532 ,  0.56585 , -0.25147 , -0.14212 ,  0.21369 , -0.2126  ,
        0.51064 , -0.98679 , -0.026492,  0.2126  ,  0.50575 , -1.078   ,
        0.40531 ,  1.0825  , -0.48179 , -0.7162  , -0.095467,  0.016159,
       -0.20646 ,  0.1386  ,  0.27936 ,  1.6731  , -0.58155 , -1.2706  ,
       -0.978   , -0.36104 ,  0.58331 ,  0.26018 ,  0.3324  ,  0.26545 ,
       -0.44384 ,  1.0533  ], dtype=float32)

In [ ]:
words=['boy','girl','cricket','man','rat','horse']

In [ ]:
sample_vectors =np.array([wv[word] for word in words])
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
result=pca.fit_transform(sample_vectors)
result

array([[-1.6330218 ,  0.25239962],
       [-2.3464072 ,  0.0123672 ],
       [ 4.0722127 ,  1.2440364 ],
       [-1.2357395 ,  2.1143804 ],
       [ 0.46382365, -2.0837767 ],
       [ 0.67913115, -1.5394082 ]], dtype=float32)

In [ ]:
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    ctr = 0
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    if ctr > 0:
        wv_res = wv_res / ctr
    return wv_res

In [ ]:
nlp=spacy.load('en_core_web_sm')
stop_words=nlp.Defaults.stop_words

In [ ]:
punctuation=string.punctuation
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
import re

def spacy_tokenizer(sentence):
    # remove numbers from the sentence
    sentence = re.sub(r'\d+', '', sentence)
    doc = nlp(sentence)
    mytokens = [word.lemma_.lower().strip() for word in doc]
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuation]
    return mytokens

In [ ]:
sent_vec('are you happy as everyone is playing')

array([ 1.48291936e-01,  5.60266614e-02,  1.60470926e-01, -6.87091403e-02,
       -8.96608115e-02,  5.31270616e-02, -4.88229862e-01,  1.36212594e-01,
       -1.08104004e-02,  2.76636666e-01, -9.82461989e-02, -2.78093933e-01,
       -3.42815003e+00, -1.57928289e-02, -2.60467768e-01, -9.46892045e-02,
       -5.22767336e-01, -1.03020446e-01, -1.21930736e-01,  7.77058344e-01,
       -2.22205698e-02, -7.31463328e-02,  2.36403137e-01,  2.44953333e-01,
        1.25094269e-01, -1.67379765e+00, -1.32929232e-01, -2.89229378e-03,
        4.61252502e-01,  6.27006999e-01, -4.19610654e-02, -2.31308668e-01,
       -1.64909648e-02,  1.09816705e-01,  5.45188541e-01,  3.57314891e-01,
       -1.30429735e-01, -3.48651671e-01, -8.32837738e-01,  4.20392001e-01,
       -2.68333670e+00,  2.70023606e-01,  2.93878466e-01, -2.19045761e-01,
        5.89272980e-02,  8.50146910e-03,  1.15886645e-02, -2.97934664e-01,
       -2.75973966e-02, -8.99837337e-01])

In [ ]:
data['tokens']=data['text'].apply(spacy_tokenizer)

In [ ]:
data.head(15)

,text,target,tokens,vec
0,Our Deeds are the Reason of this #earthquake M...,1,"[deed, reason, earthquake, allah, forgive]","[0.21803629538044333, 0.530195004120469, -0.05..."
1,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]","[-0.08332000459943499, -0.08732113827552114, -..."
2,All residents asked to 'shelter in place' are ...,1,"[resident, ask, shelter, place, notify, office...","[0.08058545670726082, 0.4348805560307069, -0.6..."
3,"13,000 people receive #wildfires evacuation or...",1,"[people, receive, wildfire, evacuation, order,...","[0.13767549162730575, 0.13255732879042625, -0...."
4,Just got sent this photo from Ruby #Alaska as ...,1,"[got, send, photo, ruby, alaska, smoke, wildfi...","[0.09356000170939499, 0.17147079860377643, -0...."
5,#RockyFire Update => California Hwy. 20 closed...,1,"[rockyfire, update, california, hwy, close, di...","[-0.000342219654056761, -0.10747766536143091, ..."
6,#flood #disaster Heavy rain causes flash flood...,1,"[flood, disaster, heavy, rain, cause, flash, f...","[-0.1457620800162355, -0.04968524910509586, -0..."
7,I'm on top of the hill and I can see a fire in...,1,"[hill, fire, wood, ...]","[-0.3917466700077057, -0.8597626474996408, 0.4..."
8,There's an emergency evacuation happening now ...,1,"[emergency, evacuation, happen, building, street]","[0.15457767862826585, 0.3410908065736294, -0.3..."
9,I'm afraid that the tornado is coming to our a...,1,"[afraid, tornado, come, area, ...]","[-0.13271500170230865, 0.21253499574959278, 0...."


In [ ]:
data['vec']=data['tokens'].apply(sent_vec)

In [ ]:
X=data['vec'].to_list()
y=data['target'].to_list()

In [ ]:
X

[array([ 0.2180363 ,  0.530195  , -0.05185226,  0.3177866 , -0.647712  ,
         0.44034919,  0.3624628 , -0.37519879, -0.575956  ,  0.63142996,
        -0.0901456 , -0.45379799, -2.94685993, -0.276578  , -0.016284  ,
         0.4626084 ,  0.13214928,  0.2859202 ,  0.109786  , -0.4817781 ,
         0.19267518,  0.270295  , -0.57944741,  0.689214  ,  0.4396924 ,
         0.47946799,  0.13971041, -0.1125948 ,  0.50799901,  0.4419166 ,
        -0.08375441,  0.03675001,  0.31727199, -0.12068999,  0.37201802,
         0.528868  , -0.64197202,  0.072056  ,  0.023548  , -0.37360285,
        -0.05002679, -0.0966    ,  0.38531401, -0.337836  , -0.32255339,
        -0.38126841,  0.066432  , -0.24331601,  0.47070202,  0.468072  ]),
 array([-8.33200046e-02, -8.73211383e-02, -4.25515683e-02, -4.54601004e-01,
        -1.61841718e-01,  8.71028591e-02,  2.44371412e-02, -1.29046718e-01,
         2.39877715e-01, -5.79827138e-01,  3.41817001e-01, -4.32125717e-01,
        -2.60682281e+00,  3.23985709e-02

In [ ]:
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()

In [ ]:
classifier.fit(X_train,y_train)

LogisticRegression()

In [ ]:
from sklearn import metrics
predicted = classifier.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.7734734077478661
Logistic Regression Precision: 0.764102564102564
Logistic Regression Recall: 0.6834862385321101
